In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [47]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        
        self.pool = nn.MaxPool2d(2,2)
        
        # Use OUTPUT_SIZE = (INPUT_SIZE-FILTER+2PADDING)/STRIDE + 1)
        # Here INPUT_SIZE AND OUTPUT_SIZE are based on dimesions of image, rather than Conv2d first and second arguments
        # 4*4 occurs because starting dimension was assumed to be 32*32
        self.fc1 = nn.Linear(64*4*4, 512)
        self.fc2 = nn.Linear(512, 7)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        
        x = x.view(-1, 64*4*4)
        
        x = F.leaky_relu(self.fc1(x))
        x = F.dropout(x, 0.25)
        x = self.fc2(x)
        
        x = F.softmax(x, dim=1)
        
        return x

model = Network()
model

Network(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1024, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=7, bias=True)
)

In [48]:
# Batch size of 5, with 3x32x32 size
input_tensor = torch.randn((5, 3, 32, 32))
input_tensor

tensor([[[[ 4.8144e-01,  1.4620e+00,  2.7872e-01,  ...,  7.5720e-01,
           -2.4793e+00,  1.1501e+00],
          [-3.1998e-01,  1.1100e-01,  8.6317e-02,  ..., -8.4137e-01,
            1.5274e-02, -1.7552e-01],
          [ 1.4694e-01,  1.9825e+00,  4.0631e-01,  ..., -7.2308e-01,
           -9.3962e-02,  8.9853e-01],
          ...,
          [-9.8358e-01,  1.6061e-01, -2.6373e+00,  ..., -3.8519e-01,
            5.7474e-01, -6.2290e-01],
          [-1.1517e+00,  7.2695e-01, -5.5362e-03,  ...,  6.6733e-03,
           -6.6562e-01,  3.7436e-01],
          [ 1.0887e-01,  1.2650e+00, -3.8918e-01,  ...,  2.4625e+00,
           -1.2666e+00,  2.3113e+00]],

         [[-6.7043e-01, -7.3248e-01,  3.1380e-01,  ..., -1.3717e+00,
            8.3985e-01,  1.2071e+00],
          [ 2.9881e+00, -8.2983e-01, -1.3414e+00,  ..., -1.2011e+00,
           -3.3211e-01,  8.3594e-01],
          [ 6.0089e-01, -9.6496e-02,  9.3702e-01,  ..., -1.1887e-01,
            1.5900e+00, -1.9815e-02],
          ...,
     

In [49]:
model(input_tensor)

tensor([[0.1547, 0.1423, 0.1438, 0.1425, 0.1388, 0.1388, 0.1390],
        [0.1520, 0.1357, 0.1442, 0.1379, 0.1335, 0.1495, 0.1472],
        [0.1620, 0.1358, 0.1554, 0.1382, 0.1343, 0.1355, 0.1389],
        [0.1645, 0.1314, 0.1445, 0.1425, 0.1318, 0.1419, 0.1434],
        [0.1510, 0.1407, 0.1499, 0.1339, 0.1397, 0.1466, 0.1381]],
       grad_fn=<SoftmaxBackward>)